In [1]:
import praw
from datetime import datetime
import math
import requests
from pathlib import Path
import json
import numpy as np

In [2]:
# read only - don't need to authenticate
reddit = praw.Reddit(
    client_id="JT_iiB-NFwFyoknGwv5fYA",
    client_secret="aVY6bitDc5BS8j4LX1cWusWjlgCaVQ",
    user_agent="mac:eecs6893Project:v1.0 (by u/Superb-Cap8523)",
)

In [3]:
# subreddit names
DALLE_SUBR = "dalle2"
MIDJ_SUBR = "midjourney"
AIART_SUBR = "aiArt"
subreddits = [DALLE_SUBR, MIDJ_SUBR, AIART_SUBR]

In [4]:
# flair for each subreddit we are scrapping
flair_dict = {MIDJ_SUBR: set(['AI Showcase - Midjourney']),
              DALLE_SUBR: set(['DALL·E 2', 'DALL·E 3']),
              AIART_SUBR: set(['Image -  I used x/ai\'s "Imagine Image and Video Generator."  ',
                            'Image - Bing Image Creator :a2:',
                            'Image - BudgetPixel',
                            'Image - BudgetPixel AI',
                            'Image - ChatGPT :a2:',
                            'Image - CivitAI\xa0:a2:',
                            'Image - ComfyUI',
                            'Image - Custom',
                            'Image - DALL E 3 :a2:',
                            'Image - DeepAI',
                            'Image - Eggie.ai',
                            'Image - Etana',
                            'Image - FLUX :a2:',
                            'Image - Gemini',
                            'Image - Google Gemini :a2:',
                            'Image - Google Imagen',
                            'Image - Grok',
                            'Image - Illustrious',
                            'Image - ImageFX.',
                            'Image - Komiko',
                            'Image - Leonardo.ai :a2:',
                            'Image - Mage.space',
                            'Image - Meta AI  :a2:',
                            'Image - Microsoft Copilot',
                            'Image - Midjourney :a2:',
                            'Image - MuleRun Agent',
                            'Image - MuleRun Halloween Costume Agent',
                            'Image - Nightcafe :a2:',
                            'Image - Other: Aierone',
                            'Image - Other: ComfyUI',
                            'Image - Other: Flat AI',
                            'Image - Other: Grok Imagine',
                            'Image - Other: Hailou Ai',
                            'Image - Other: Moescape [Illustrious]',
                            'Image - Other: MuleRunAI',
                            'Image - Other: NovelAI',
                            'Image - Other: PixNova AI',
                            'Image - Other: Please edit, or your post may be deleted.',
                            'Image - Other: Seedream 4.0',
                            'Image - Other: Wan/Grok/Meta/Nano Banana/Qwen',
                            'Image - Other: Whisk',
                            'Image - Qwen',
                            'Image - Seedream',
                            'Image - Seedream 1.0',
                            'Image - Sogni AI',
                            'Image - SoraAI :a2:',
                            'Image - Stable Diffusion + Manual Editing',
                            'Image - Stable Diffusion :a2:',
                            'Image - String AI',
                            'Image - VQGAN+clip',
                            'Image - Wan',
                            'Image - Whisk',
                            'Image - etana'])
            }

In [6]:
# check the timeframe of 1000 posts so that we can get how many posts to expect to scrape per day later
# but note that this isn't even filtered, so its an overestimate

for sub_name in subreddits:
    subreddit = reddit.subreddit(sub_name)
    posts = list(subreddit.new(limit=1000))
    
    newest = datetime.fromtimestamp(posts[0].created_utc)
    oldest = datetime.fromtimestamp(posts[-1].created_utc)
    time_span = newest - oldest
    
    total_seconds = time_span.total_seconds()
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)

    print(f"r/{sub_name}:")
    print(f"oldest: {oldest}")
    print(f"newest: {newest}")
    print(f"time span: {time_span.days} days")
    ppd = 1000/time_span.days
    print(f"approximate amount of posts/day: {ppd} posts/day (all the posts)\n")

r/dalle2:
oldest: 2023-09-30 07:29:43
newest: 2025-11-18 05:04:10
time span: 779 days
approximate amount of posts/day: 1.2836970474967908 posts/day (all the posts)

r/midjourney:
oldest: 2025-10-22 11:34:36
newest: 2025-11-20 10:31:35
time span: 28 days
approximate amount of posts/day: 35.714285714285715 posts/day (all the posts)

r/aiArt:
oldest: 2025-11-10 14:33:39
newest: 2025-11-20 10:32:22
time span: 9 days
approximate amount of posts/day: 111.11111111111111 posts/day (all the posts)



In [9]:
# get and save images from the given subreddit and flair INITIALLY
#! ONLY RUN THIS ONCE
def save_imgs_from_subreddit_init(subr, flair):
    # save metadata for aiArt
    metadata = {}

    subreddit = reddit.subreddit(subr)
    img_cnt = 0

    for submission in subreddit.new(limit=1000):

        # only get flairs related to images
        if submission.link_flair_text not in flair:
            continue

        # check if the url is an image
        if not submission.url.endswith(('.jpg', '.jpeg', '.png')):
            continue

        # skip nsfw imgs
        if submission.over_18:
            continue

        # get file extension
        file_extension = submission.url.split(sep=".")[-1].lower()

        # get folder for subreddit
        image_dir = Path(f"data/{subr}")
        image_dir.mkdir(exist_ok=True, parents=True)

        # download the image to inital data folder
        try:
            response = requests.get(submission.url, timeout=10)
            filename = image_dir / f"img_{subr}_{submission.id}.{file_extension}"

            with open(filename, "wb") as f:
                f.write(response.content)

            img_cnt +=1

            # only the aiArt subreddit needs metadata
            if subr != MIDJ_SUBR:
                metadata[str(filename)] = {'flair': submission.link_flair_text,
                                      'url': submission.url}
                
        except Exception as e:
            print(f"error {e} for {submission.url}")

    # end of for loop, if its aiArt, need to save the metadata also to the folder
    if subr != MIDJ_SUBR:
        metadata_filepath = f"{image_dir}/metadata.json"
        with open(metadata_filepath, "w") as json_file:
            json.dump(metadata, json_file, indent=4)

    print(f"for subreddit {subr}, saved {img_cnt} images from past 1000 posts\n")


In [ ]:
# save images from past 1k post on 11/9 for all subreddits
for subr in subreddits:
    save_imgs_from_subreddit_init(subr, flair_dict[subr])

for subreddit dalle2, saved 291 images from past 1000 posts

for subreddit midjourney, saved 192 images from past 1000 posts

for subreddit aiArt, saved 482 images from past 1000 posts



In [27]:
# number of image post per day
filtered_ppd = np.array([291, 192, 482])/np.array([767, 29, 9])
for subreddit, ppd in zip(subreddits, filtered_ppd):
    print(f"for r/{subreddit}, {ppd} image posts per day")

for r/dalle2, 0.379400260756193 image posts per day
for r/midjourney, 6.620689655172414 image posts per day
for r/aiArt, 53.55555555555556 image posts per day
